### Q: Write a program for policy iteration and re-solve Jack’s car rental problem with the following changes. One of Jack’s employees at the first locationrides a bus home each night and lives near the second location. She is happy to shuttleone car to the second location for free. Each additional car still costs 2, as do all carsmoved in the other direction. In addition, Jack has limited parking space at each location. If more than 10 cars are kept overnight at a location (after any moving of cars), then an additional cost of 4 must be incurred to use a second parking lot (independent of how many cars are kept there). These sorts of nonlinearities and arbitrary dynamics often occur in real problems and cannot easily be handled by optimization methods other than dynamic programming. To check your program, first replicate the results given for the original problem.

In [ ]:
import numpy as np
from copy import copy
from random import choice
from scipy.stats import poisson
from itertools import product, chain

import matplotlib.pyplot as plt

**Constantes e definições**

In [ ]:
# Constantes
RENT_CAR_REWARD = 10
MOVE_CAR_COST = 2
MAX_MOVE_CARS = 5
TOTAL_CARS_BY_LOCATION = 20

FIRST_LOCATION_FREE_MOVE_CARS = 0
SECOND_LOCATION_FREE_MOVE_CARS = 0

PARKING_LOT_COST = 0
FREE_PARKING_SPACE = TOTAL_CARS_BY_LOCATION

# λ para a distribuição de probabilidade Poisson do número de retornos e pedidos de aluguel
FIRST_LOCATION_REQUEST_POISSON_LAMBDA = 3
FIRST_LOCATION_RETURN_POISSON_LAMBDA = 3
SECOND_LOCATION_REQUEST_POISSON_LAMBDA = 4
SECOND_LOCATION_RETURN_POISSON_LAMBDA = 2

# Hyperparâmetros 
DISCOUNT_FACTOR = 0.9
ERROR_THRESHOLD = 0.01
MAX_POLICY_IMPROVEMENT_ITERATIONS = 10

In [ ]:
# Estados
STATES_BY_LOCATION = tuple(range(TOTAL_CARS_BY_LOCATION + 1))
STATES = list(product(STATES_BY_LOCATION, STATES_BY_LOCATION))

In [ ]:
def actions_set(state):
    """
    Ações possíveis dado um estado. Exemplo: Dado estado (15, 18) -> [-5, -4, -3, -2, -1, 0, 1, 2]
    """
    first_location, second_location = state
    # O número máximo de carros que pode sair de uma locadora é, o menor número entre:
    #    - O número de carros na locadora (first_location)
    #    - O número máximo de carros que a segunda locadora pode receber (TOTAL_CARS_BY_LOCATION - second_location)
    #    - O número máximo de movimentos (MAX_MOVE_CARS)
    from_first_location = min(first_location, TOTAL_CARS_BY_LOCATION - second_location, MAX_MOVE_CARS)
    from_first_location = range(1, from_first_location + 1)
    from_second_location = min(second_location, TOTAL_CARS_BY_LOCATION - first_location, MAX_MOVE_CARS)
    from_second_location = range(-1, from_second_location * -1 - 1, -1)
    return tuple(chain(from_second_location, (0,), from_first_location))

# Dicionários com todas as ações possíveis para cada estado
ACTIONS_SET = {s: actions_set(s) for s in STATES}

In [ ]:
# Não usado para o problema, a função está aqui por pena de apagar
def rental_cars_probability_given_balance(rental_cars, balance, request_lambda, return_lambda):
    """
    Probabilidade de pedidos (rental_cars) dado o saldo final de carros (balance).
    Segundo o teorema de Bayes (Probabilidade condicional), temos:
    P(A | B) = P(B | A) * P (A) / P(B)
    P(A) -> Probabilidade da quantidade de alugueis no dia
    P(B) -> Probabilidade do saldo final de carros no dia
    P(B | A) -> Probabilidade do saldo final de carros dado a quantidade de alugueis no dia
    
    Dado a quantidade de alugueis no dia, a probabilidade do saldo final de carros é igual a probabilidade
    de carros devolvidos = saldos + quantidade de carros alugados no dia (balance + rental_cars).
    
    A probabilidade do saldo final de carros é dado pela distribuição Skellam. Skellam é a 
    dsitribuição de probabilidade da diferença entre duas variáveis aleatórias de Poission
    """
    return (poisson.pmf(balance + rental_cars, return_lambda) * poisson.pmf(rental_cars, request_lambda) / 
            skellam.pmf(balance, return_lambda, request_lambda))

In [ ]:
def reward_expectation(state, action, state_values):
    """
    Recompensa total esperada dado um estado e uma ação
    """
    first_location, second_location = state

    # Primeiro é necessário fazer a transferência de carros entre as locadoras
    first_location -= action
    second_location += action
    
    # Custo por está fazendo movimentação dos carros entre as locadoras
    move_car_reward = 0
    if action > 0:
        move_car_reward = (action - FIRST_LOCATION_FREE_MOVE_CARS) * -MOVE_CAR_COST
    elif action < 0:
        move_car_reward = (abs(action) - SECOND_LOCATION_FREE_MOVE_CARS) * -MOVE_CAR_COST
    
    reward = 0
    
    for first_rented in range(first_location + 1):
        # A probabilidade de todos os carros da locadoras serem alugados inclui também 
        # as probabilidades dos números maiores que a quantidade total de carros. Pois se a quantidade
        # de pedidos exceder, ainda sim o todos os carros vão ser alugados. Ou seja é a probabilidade acumulada
        # até ali subtraída de 100%. Esse pensamento é aplicado para todos os eventos Poisson desse problema.
        if first_rented == first_location:
            rent_probability_1 = 1 - poisson.cdf(first_rented - 1, FIRST_LOCATION_REQUEST_POISSON_LAMBDA)
        else:
            rent_probability_1 = poisson.pmf(first_rented, FIRST_LOCATION_REQUEST_POISSON_LAMBDA)
        
        first_reward = first_rented * RENT_CAR_REWARD
        
        for second_rented in range(second_location + 1):
            if second_rented == second_location:
                rent_probability = 1 - poisson.cdf(second_rented - 1, SECOND_LOCATION_REQUEST_POISSON_LAMBDA)
            else:
                rent_probability = poisson.pmf(second_rented, SECOND_LOCATION_REQUEST_POISSON_LAMBDA)
            
            rent_probability *= rent_probability_1
            
            rent_reward = first_reward + second_rented * RENT_CAR_REWARD
    
            # Porque TOTAL_CARS_BY_LOCATION - first_location + first_rented?
            # A quantidade de carros retornados não deve passar capacidade máxima de carros da locadora.
            # Os estados além da capacidade não devem ser mapeados.
            for first_return in range(TOTAL_CARS_BY_LOCATION - first_location + first_rented + 1):
                if first_return == TOTAL_CARS_BY_LOCATION - first_location + first_rented:
                    return_probability_1 = 1 - poisson.cdf(first_return - 1, FIRST_LOCATION_RETURN_POISSON_LAMBDA)
                else:    
                    return_probability_1 = poisson.pmf(first_return, FIRST_LOCATION_RETURN_POISSON_LAMBDA)
                
                for second_return in range(TOTAL_CARS_BY_LOCATION - second_location + second_rented + 1):
                    if second_return == TOTAL_CARS_BY_LOCATION - second_location + second_rented:
                        return_probability = 1 - poisson.cdf(second_return - 1, SECOND_LOCATION_RETURN_POISSON_LAMBDA)
                    else:    
                        return_probability = poisson.pmf(second_return, SECOND_LOCATION_RETURN_POISSON_LAMBDA)
                    
                    return_probability *= return_probability_1
                    
                    next_first_location = first_location - first_rented + first_return
                    next_second_location = second_location - second_rented + second_return
                    next_state = next_first_location, next_second_location
                    
                    parking_lot_reward = 0 if next_first_location <= FREE_PARKING_SPACE else -PARKING_LOT_COST
                    parking_lot_reward += 0 if next_second_location <= FREE_PARKING_SPACE else -PARKING_LOT_COST
                    
                    future_reward = DISCOUNT_FACTOR * state_values[next_state]
                    
                    reward += rent_probability * return_probability * (move_car_reward + rent_reward 
                                                                       + parking_lot_reward + future_reward)

    return reward

In [ ]:
def policy_evaluation(state_values, policy):
    """
    Avaliação da política: atualizando os valor reais de cada estado dado uma política.
    """
    error = None
    while error is None or error > ERROR_THRESHOLD:
        error = 0
        for s in STATES:
            value = state_values[s]
            state_values[s] = reward_expectation(s, policy[s], state_values)
            error = max(error, abs(value - state_values[s]))
        print(f'Policy evaluation error: {error}')

In [ ]:
def policy_improvement(state_values, policy):
    """
    Melhora a política utilizando a função state-value como base. Escolhe as ações que vão dá as melhores recompensas.
    :return True se a política foi alterada. Caso contrário, False, significando que política já está estável. 
    """
    is_policy_stable = True
    for s in STATES:
        old_action = policy[s]
        policy[s] = max(ACTIONS_SET[s], key=lambda a: reward_expectation(s, a, state_values))
        is_policy_stable = False if old_action != policy[s] else is_policy_stable
    return not is_policy_stable

In [ ]:
def plot_results(policy, state_values):
    """
    Plota o gráfico color map para a política e uma projeção do 3D da função estado-valor
    """
    
    # Plot do Color Map
    data = np.array([[policy[(f, s)] for s in range(TOTAL_CARS_BY_LOCATION + 1)] for f in range(TOTAL_CARS_BY_LOCATION + 1)])

    fig = plt.figure(figsize=(10, 4))
    color_map_axis = fig.add_subplot(1, 2, 1)
    color_map = color_map_axis.imshow(data, origin='lower', interpolation='none', vmin=-MAX_MOVE_CARS, vmax=MAX_MOVE_CARS)
    color_map_axis.set_xlabel('#Cars at second location', fontsize=13)
    color_map_axis.set_ylabel('#Cars at first location', fontsize=13)
    color_map_axis.set_title('Jack\'s car rental problem policy')
    plt.colorbar(color_map, ticks=[c for c in range(-MAX_MOVE_CARS, MAX_MOVE_CARS + 1)])

    # Plot do Estado-Valor
    state_values_data = np.array([[state_values[(f, s)] for s in range(TOTAL_CARS_BY_LOCATION + 1)] for f in range(TOTAL_CARS_BY_LOCATION + 1)])
    state_value_axis = fig.add_subplot(1, 2, 2, projection='3d')

    X = np.arange(TOTAL_CARS_BY_LOCATION + 1)
    Y = np.arange(TOTAL_CARS_BY_LOCATION + 1)
    X, Y = np.meshgrid(X, Y)

    state_value_axis.plot_surface(X, Y, state_values_data)
    state_value_axis.set_title('State-value function vπ(s)')
    state_value_axis.set_xlabel('#Cars at second location', fontsize=11)
    state_value_axis.set_ylabel('#Cars at first location', fontsize=11)
    state_value_axis.set_zlabel('vπ(s)', fontsize=11)

**Problema original**

In [ ]:
# Inicialização do vetor state-value e policy
state_values = {s: 0 for s in STATES}
policy = {s: choice(ACTIONS_SET[s]) for s in STATES}

In [ ]:
is_policy_stable = False
for i in range(MAX_POLICY_IMPROVEMENT_ITERATIONS):
    print(f'Iteration {i + 1}: Policy evaluation...')
    policy_evaluation(state_values, policy)
    print(f'Iteration {i + 1}: Policy improvement...')
    if not policy_improvement(state_values, policy):
        break
print('Finished!')

In [ ]:
plot_results(policy, state_values)

**Problema Modificado**
- Transportar carros da primeira locadora para a segunda tem frete grátis para somente um carro. Transportar mais de um tem custo de 2 para cada carro adicional.
- Há um custo de 4 para manter mais de 10 carros nas locadoras.

In [ ]:
# Limpar o cache do problema anterior
REWARD_EXPECTATION_CONSTANTS_CACHE = dict()

# Definição das constantes
FIRST_LOCATION_FREE_MOVE_CARS = 1
SECOND_LOCATION_FREE_MOVE_CARS = 0

PARKING_LOT_COST = 4
FREE_PARKING_SPACE = 10

In [ ]:
# Inicialização do vetor state-value e policy
state_values_2 = {s: 0 for s in STATES}
policy_2 = {s: choice(ACTIONS_SET[s]) for s in STATES}

In [ ]:
is_policy_stable = False
for i in range(MAX_POLICY_IMPROVEMENT_ITERATIONS):
    print(f'Iteration {i + 1}: Policy evaluation...')
    policy_evaluation(state_values_2, policy_2)
    print(f'Iteration {i + 1}: Policy improvement...')
    if not policy_improvement(state_values_2, policy_2):
        break
print('Finished!')

In [ ]:
plot_results(policy_2, state_values_2)